In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import contextlib
import gc
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Lambda, Dropout
from keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from labels import class_labels

In [3]:
train_dir     = 'train' 
val_dir       = 'val'
batch_size    = 64
img_size      = (224,224)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            seed=123,
                                                            batch_size=batch_size,
                                                            image_size=img_size)

val_dataset   = tf.keras.utils.image_dataset_from_directory(val_dir,
                                                                 shuffle=True,
                                                                 seed=123,
                                                                 batch_size=batch_size,
                                                                 image_size=img_size)

Found 9469 files belonging to 10 classes.


2024-10-20 17:37:53.323552: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 3925 files belonging to 10 classes.


In [4]:
class_labels

['tench',
 'english springer',
 'cassette player',
 'chain saw',
 'church',
 'french horn',
 'garbage truck',
 'gas pump',
 'golf ball',
 'parachute']

In [5]:
#normalize data
normalization_layer = tf.keras.layers.Rescaling(scale=1./255)
train_dataset       = train_dataset.map(lambda x, y:(normalization_layer(x), tf.one_hot(y, depth=10)))
val_dataset         = val_dataset.map(lambda x, y:(normalization_layer(x),  tf.one_hot(y, depth=10)))

image_batch, _      = next(iter(train_dataset))
first_image         = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [6]:
for image, labels in train_dataset.take(1):
    print("Batch of images shape:", image.shape)
    print("Batch of labels shape:" ,labels.shape)

Batch of images shape: (64, 224, 224, 3)
Batch of labels shape: (64, 10)


In [7]:
val_batches  = tf.data.experimental.cardinality(val_dataset) #calculates the num of batches in the validation set
test_dataset = val_dataset.take(val_batches // 5) # take the first 20% of the val dataset to create the new test set
val_dataset  = val_dataset.skip(val_batches // 5) # skip the first 20% of the val dataset to keep remaining 80% as the new val set.

print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_dataset))
print("number of images in validation set :", len(val_dataset)*batch_size)
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))
print("number of images in test set :", len(test_dataset)*batch_size)

Number of validation batches: 50
number of images in validation set : 3200
Number of test batches: 12
number of images in test set : 768


In [8]:
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.backend import epsilon

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [10]:
import random

def set_random_seeds(seed_value):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)

In [11]:
def create_model():
    base_model = tf.keras.applications.DenseNet121(include_top = False,
                                                   input_shape = (224,224,3),
                                                   weights     = 'imagenet',)
    base_model.trainable = False

    x       = base_model.output
    x       = GlobalAveragePooling2D()(x)
    x       = Dropout(0.2)(x)
    x       = Dense(512, activation ='relu')(x)
    x       = Dropout(0.4)(x)
    outputs = Dense(10, activation  ='softmax')(x)

    model   = tf.keras.Model(inputs = base_model.input, outputs=outputs)
    
    
    model.compile(optimizer=Adam(),
                 loss    = 'categorical_crossentropy',
                 metrics = ['accuracy',
                         Precision(name = 'precision'),
                         Recall(name    = 'recall'),
                         AUC(name       = 'auc'),
                         F1Score(name   = 'f1_score')])
    return model

In [8]:
num_trials  = 10
results     = []

# Create directory if it doesn't exist
save_dir    = "DenseNet_models"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for i in range(num_trials):
    print(f"Trial {i+1}")
    seed    = i  # Different seed for each trial
    set_random_seeds(seed)

    model   = create_model()
    
    history = model.fit(train_dataset, validation_data=val_dataset, epochs=10, verbose=2)

    val_loss, val_acc, _, _, _, _ = model.evaluate(val_dataset)
    print(f"Validation Accuracy for trial {i+1}: {val_acc}")
    
    results.append({
        'trial'   : i+1,
        'seed'    : seed,
        'val_loss': val_loss,
        'val_acc' : val_acc,
        'history' : history.history
    })

    test_loss, test_acc, _, _, _, _ = model.evaluate(test_dataset)
    print(f"Test Accuracy for trial {i+1}: {test_acc}")
    
    model_name = f'DenseNet{i+1}.h5'
    model_path = os.path.join(save_dir, model_name)

    # Save the model if both validation and test accuracy are greater than 0.95
    if val_acc > 0.95 and test_acc > 0.95:
        model.save(model_path)
        print(f"Model saved as {model_path}")

Trial 4
Epoch 1/10


/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:2699: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


148/148 - 56s - loss: 0.2575 - accuracy: 0.9222 - precision: 0.9575 - recall: 0.8997 - auc: 0.9942 - f1_score: 0.9277 - val_loss: 0.0608 - val_accuracy: 0.9772 - val_precision: 0.9843 - val_recall: 0.9747 - val_auc: 0.9995 - val_f1_score: 0.9795 - 56s/epoch - 382ms/step
Epoch 2/10
148/148 - 47s - loss: 0.0966 - accuracy: 0.9673 - precision: 0.9735 - recall: 0.9637 - auc: 0.9985 - f1_score: 0.9686 - val_loss: 0.0525 - val_accuracy: 0.9829 - val_precision: 0.9863 - val_recall: 0.9804 - val_auc: 0.9995 - val_f1_score: 0.9833 - 47s/epoch - 320ms/step
Epoch 3/10
148/148 - 48s - loss: 0.0754 - accuracy: 0.9747 - precision: 0.9793 - recall: 0.9729 - auc: 0.9989 - f1_score: 0.9761 - val_loss: 0.0559 - val_accuracy: 0.9810 - val_precision: 0.9847 - val_recall: 0.9794 - val_auc: 0.9997 - val_f1_score: 0.9821 - 48s/epoch - 322ms/step
Epoch 4/10
148/148 - 51s - loss: 0.0649 - accuracy: 0.9787 - precision: 0.9816 - recall: 0.9759 - auc: 0.9991 - f1_score: 0.9788 - val_loss: 0.0541 - val_accuracy: 0

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as DenseNet_models/DenseNet4.h5
Trial 5
Epoch 1/10
148/148 - 54s - loss: 0.2516 - accuracy: 0.9232 - precision: 0.9606 - recall: 0.9045 - auc: 0.9949 - f1_score: 0.9317 - val_loss: 0.0806 - val_accuracy: 0.9759 - val_precision: 0.9798 - val_recall: 0.9702 - val_auc: 0.9990 - val_f1_score: 0.9750 - 54s/epoch - 365ms/step
Epoch 2/10
148/148 - 50s - loss: 0.1020 - accuracy: 0.9659 - precision: 0.9734 - recall: 0.9621 - auc: 0.9985 - f1_score: 0.9677 - val_loss: 0.0586 - val_accuracy: 0.9845 - val_precision: 0.9888 - val_recall: 0.9797 - val_auc: 0.9993 - val_f1_score: 0.9842 - 50s/epoch - 336ms/step
Epoch 3/10
148/148 - 50s - loss: 0.0724 - accuracy: 0.9768 - precision: 0.9804 - recall: 0.9741 - auc: 0.9992 - f1_score: 0.9773 - val_loss: 0.0517 - val_accuracy: 0.9842 - val_precision: 0.9866 - val_recall: 0.9813 - val_auc: 0.9992 - val_f1_score: 0.9840 - 50s/epoch - 338ms/step
Epoch 4/10
148/148 - 46s - loss: 0.0673 - accuracy: 0.9788 - precision: 0.9818 - recall: 0.9761 - auc:

In [11]:
for result in results:
    print(f"Trail {result['trial']}: Seed {result['seed']} - Validation Accuracy: {result['val_acc']}")

Trail 4: Seed 3 - Validation Accuracy: 0.9816281199455261
Trail 5: Seed 4 - Validation Accuracy: 0.9838454127311707
Trail 6: Seed 5 - Validation Accuracy: 0.9781438112258911
Trail 7: Seed 6 - Validation Accuracy: 0.9838454127311707
Trail 8: Seed 7 - Validation Accuracy: 0.9882799983024597
Trail 9: Seed 8 - Validation Accuracy: 0.9854292273521423
Trail 10: Seed 9 - Validation Accuracy: 0.9813113808631897
